## 3-Layer Model with Informtion, Behavior, Disease

In [ ]:
#!pip install hypernetx

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import hypernetx as hnx

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import math as math
from math import log
import seaborn as sns
import pandas as pd
import matplotlib.ticker as ticker


## Part 1: Hypergraph Generation
The following steps generate a hyper graph using the XGI/HyperNetX python package,  following power-law degree distribution for predifined number of nodes n, number of hyperedges num_hyper_edges, degree exponent gamma, using a configuration model with data stored in a dictionary.

In [4]:
# Step 1: Generate Degree Sequence
def generate_degree_sequence(n, gamma, kmin):
    # Generate a random set from the power law distribution
    u = np.random.uniform(size=n)
    degrees = np.ceil((1.0 - u) ** (-1.0 / (gamma - 1.0)))

    # Adjust degrees based on the minimum and maximum degree values
    kmax = int(np.sqrt(n))
    #kmax = int(((gamma-1)/(gamma-2) * n )** (1/gamma))  # max degree
    degrees = degrees[(degrees >= kmin) & (degrees <= kmax)].astype(int)

    # Truncate or pad the sequence to match the length specified
    if len(degrees) >= n:
        degrees = degrees[:n]
    else:
        degrees = np.concatenate((degrees, np.full(n - len(degrees), kmin)))

    return degrees.tolist()

# Step 2: Generate Hyper Edge Size Sequence
def generate_hyper_edge_sizes(degrees, num_hyper_edges):
    total_degrees = sum(degrees)
    hyper_edge_sizes = []

    # Calculate the average size for each hyper edge
    avg_size = total_degrees // num_hyper_edges
    remainder = total_degrees % num_hyper_edges

    # Define the range for the random distribution
    min_size = 2  # Lower bound of the range
    max_size = int(np.sqrt(total_degrees))  # Upper bound of the range
    #max_size = len(degrees) - num_hyper_edges  # Upper bound of the range

    # Generate hyper edge sizes
    for _ in range(num_hyper_edges):
        size = random.randint(min_size, max_size)
        hyper_edge_sizes.append(size)

    return hyper_edge_sizes


# Step 3: Create Copies of Nodes
def create_node_copies(degrees):
    node_copies = []
    for i, degree in enumerate(degrees):
        for _ in range(degree):
            node_copies.append(i)
    return node_copies

# Step 4: Create Copies of Hyper Edges
def create_hyper_edge_copies(hyper_edge_sizes):
    hyper_edge_copies = []
    for i, size in enumerate(hyper_edge_sizes):
        for _ in range(size):
            hyper_edge_copies.append(i)
    return hyper_edge_copies

# Step 5: Randomly Pair Copies without Repeated Pairs
def randomly_pair_copies(node_copies, hyper_edge_copies):
    pairs = []
    paired_hyper_edges = {} # Using a dictionary to track paired hyper-edges with nodes

    for node_copy in node_copies:
        available_hyper_edges = [h for h in hyper_edge_copies if (h, node_copy) not in paired_hyper_edges]

        # If no available hyper-edges left, shuffle the paired hyper-edges and reset
        if not available_hyper_edges:
            paired_hyper_edges = {}
            available_hyper_edges = [h for h in hyper_edge_copies if (h, node_copy) not in paired_hyper_edges]

        # Randomly choose a hyper-edge that has not been paired yet with the current node
        chosen_hyper_edge = random.choice(available_hyper_edges)
        pairs.append((node_copy, chosen_hyper_edge))

        # Add to paired_hyper_edges
        paired_hyper_edges[(chosen_hyper_edge, node_copy)] = True
        hyper_edge_copies.remove(chosen_hyper_edge)

    return pairs

# Step 6: Convert Bipartite Graph to A Hypergraph Dictionary
def convert_to_hypergraph(pairs):
    hypergraph = {}
    for pair in pairs:
        node, hyper_edge = pair
        if hyper_edge in hypergraph:
            hypergraph[hyper_edge].append(node)
        else:
            hypergraph[hyper_edge] = [node]
    return hypergraph


In [5]:
def build_hypergraph(n, gamma, kmin, num_hyper_edges):
    # Step 1: Generate Degree Sequence
    degrees = generate_degree_sequence(n, gamma, kmin)
    print("Degree Sequence: ", degrees)

    # Step 2: Generate Hyper Edge Size Sequence
    hyper_edge_sizes = generate_hyper_edge_sizes(degrees, num_hyper_edges)
    print("Hyper Edge Sizes: ", hyper_edge_sizes)

    # Step 3: Create Copies of Nodes
    node_copies = create_node_copies(degrees)

    # Step 4: Create Copies of Hyper Edges
    hyper_edge_copies = create_hyper_edge_copies(hyper_edge_sizes)

    # Step 5: Randomly Pair Copies
    pairs = randomly_pair_copies(node_copies, hyper_edge_copies)

    # Step 6: Convert Bipartite Graph to Hypergraph
    hyperedge_dict = convert_to_hypergraph(pairs)

    # Print the resulting hypergraph
    print("Hypergraph Dictionary: ", hyperedge_dict)

    return degrees, hyperedge_dict

In [6]:
# Test 2
n2 = 500  # Number of nodes
gamma2 = 2.5  # Power-law exponent
kmin2 = 3  # Minimum degree
num_hyper_edges2 = 450  # Desired number of hyper edges

degrees2, hyperedge_dict2 = build_hypergraph(n2, gamma2, kmin2, num_hyper_edges2)
H2 = hnx.Hypergraph(hyperedge_dict2)

Degree Sequence:  [3, 3, 3, 16, 3, 3, 10, 4, 3, 6, 4, 4, 6, 9, 4, 4, 3, 4, 5, 9, 5, 5, 4, 3, 3, 3, 7, 7, 3, 6, 5, 10, 6, 3, 10, 6, 8, 6, 5, 5, 4, 6, 6, 6, 3, 3, 4, 3, 5, 3, 4, 3, 6, 4, 3, 7, 4, 3, 5, 15, 3, 3, 4, 3, 5, 3, 3, 6, 3, 3, 5, 12, 3, 3, 3, 9, 12, 4, 10, 3, 3, 8, 4, 4, 4, 3, 3, 3, 8, 3, 3, 3, 12, 3, 7, 3, 6, 9, 3, 9, 8, 3, 6, 3, 5, 4, 4, 8, 3, 7, 3, 4, 4, 9, 3, 3, 10, 5, 3, 3, 3, 11, 8, 3, 5, 3, 3, 3, 4, 7, 3, 18, 7, 4, 4, 15, 3, 3, 3, 8, 3, 3, 3, 3, 3, 13, 5, 3, 5, 3, 6, 6, 5, 3, 3, 3, 8, 3, 3, 3, 3, 3, 4, 13, 4, 3, 3, 13, 5, 4, 3, 6, 4, 3, 3, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

## Part 2: Assign Behavior Status
NP represents the state of no protection, while P represents the state of with protection.

In [7]:
def assign_protection(hypergraph, fraction_protected):
    # Get the list of nodes from the hypergraph
    nodes = list(hypergraph.nodes())

    # Calculate the number of nodes to protect
    num_nodes_to_protect = int(len(nodes) * fraction_protected)

    # Randomly choose nodes to protect
    nodes_to_protect = random.sample(nodes, num_nodes_to_protect)

    # Initialize the protection status dictionary
    protection_status = {}

    # Assign protection status to each node
    for node in nodes:
        if node in nodes_to_protect:
            protection_status[node] = "V"  # vaccinated node
        else:
            protection_status[node] = "NV"  # Non-vaccinated node

    #print(protection_status)

    return protection_status

In [8]:
# Test:
fraction_protected = 0.02
protection_status_dict = assign_protection(H2, fraction_protected)
print(protection_status_dict)

{np.int64(0): 'NV', np.int64(201): 'NV', np.int64(210): 'NV', np.int64(245): 'NV', np.int64(284): 'NV', np.int64(345): 'NV', np.int64(440): 'NV', np.int64(37): 'NV', np.int64(47): 'NV', np.int64(126): 'NV', np.int64(443): 'NV', np.int64(463): 'NV', np.int64(55): 'NV', np.int64(110): 'NV', np.int64(164): 'NV', np.int64(1): 'NV', np.int64(281): 'NV', np.int64(469): 'NV', np.int64(497): 'NV', np.int64(167): 'NV', np.int64(344): 'NV', np.int64(75): 'NV', np.int64(132): 'NV', np.int64(175): 'NV', np.int64(185): 'NV', np.int64(387): 'NV', np.int64(439): 'NV', np.int64(2): 'NV', np.int64(347): 'NV', np.int64(477): 'NV', np.int64(17): 'NV', np.int64(73): 'NV', np.int64(107): 'NV', np.int64(193): 'NV', np.int64(235): 'NV', np.int64(352): 'NV', np.int64(402): 'NV', np.int64(462): 'NV', np.int64(3): 'NV', np.int64(286): 'NV', np.int64(137): 'NV', np.int64(211): 'NV', np.int64(14): 'NV', np.int64(121): 'NV', np.int64(148): 'NV', np.int64(195): 'NV', np.int64(226): 'NV', np.int64(233): 'NV', np.int


## Part 3: Assign Threshold
The following steps assigns a threshold value to each node in the network. The threshold follows a uniform or normal distribution with predefined mean (mu) and standard deviation (sigma).

In [9]:
# Defines the parameters to be used
mu = 0.1
sigma = 0.05

# Function to assign thresholds to the individual nodes
def assign_thresholds(hypergraph, mu, sigma):
    NV = hypergraph.order()
    Ltre = {}

    for node in hypergraph.nodes():
          # Uniform distribution: #
          #Ltre[node] = np.random.uniform()
          # Normal distrution
          while True:
              threshold = random.gauss(mu, sigma)
              if 0 < threshold < 1:
                  break
          Ltre[node] = threshold

    return Ltre

In [10]:
Ltre2 = assign_thresholds(H2, mu, sigma)

print("Threshold List for Nodes: ", Ltre2 )

Threshold List for Nodes:  {np.int64(0): 0.1441425423174106, np.int64(201): 0.1665965284675476, np.int64(210): 0.13825189541369215, np.int64(245): 0.12103025675605308, np.int64(284): 0.05739812247661965, np.int64(345): 0.0880302414507447, np.int64(440): 0.1401048191603669, np.int64(37): 0.13006499473656347, np.int64(47): 0.04915319560498116, np.int64(126): 0.07740020706750442, np.int64(443): 0.17627037371008744, np.int64(463): 0.048740078130692636, np.int64(55): 0.0910733338243857, np.int64(110): 0.05072287085393428, np.int64(164): 0.10843833903254775, np.int64(1): 0.06871958899708006, np.int64(281): 0.14102168816277733, np.int64(469): 0.07929084411099815, np.int64(497): 0.07583440125784152, np.int64(167): 0.05555346816932075, np.int64(344): 0.1072447696994364, np.int64(75): 0.07845702690309989, np.int64(132): 0.1517439299309062, np.int64(175): 0.11606906968605005, np.int64(185): 0.10711929640111278, np.int64(387): 0.1743235047560489, np.int64(439): 0.13922133460444472, np.int64(2): 0.

# Part 4: The ICE Model (The Information Cognition Epidemics Model)
## Information Layer
The misinformation spread occurs on a hyperedge network involving group spreading. The three stages are U(unaware), G(gossip/spreader), and C(stifler/corrected).  

## Cognition Layer
In the cognitive behavioral layer, P is protected, and N is not protected. The rate of transition from state P to N, p, depends on the information layer. The rate from NP to P is 1-p. The transition rate of a node is also affected by the number of active spreader/stiflers. The bigger number of active neighbors, the faster the rate. Another way behavior may change is based on the fraction of protected neighbors.

## Epidemics Layer
In the epidemics layer, the possible disease states are S(susceptible), I(infected), and R(recovered). The illness spreading is pairwise. The disease propagation rate depends on the fraction of protected individuals $\rho_P$.



In [11]:
n = 500  # Number of nodes

# Information Layer
gamma_i = 2.5  # Power-law exponent
kmin_i = 3  # Minimum degree
num_hyper_edges_i = 450  # Desired number of hyper edges
ldeg_i, hyperedge_dict_i = build_hypergraph(n, gamma_i, kmin_i, num_hyper_edges_i)
inw = hnx.Hypergraph(hyperedge_dict_i)
ltre = assign_thresholds(inw, 0.10, 0.05)
print("Acceptance Threshold Sequence: ", ltre)

Degree Sequence:  [3, 3, 5, 3, 3, 3, 4, 3, 4, 6, 5, 5, 4, 6, 3, 3, 3, 4, 18, 5, 6, 4, 3, 3, 3, 22, 4, 3, 3, 3, 4, 6, 3, 5, 3, 3, 5, 11, 3, 3, 3, 3, 7, 3, 6, 4, 4, 3, 6, 10, 3, 9, 9, 5, 7, 7, 3, 3, 4, 5, 9, 13, 3, 3, 4, 3, 3, 3, 3, 9, 3, 4, 4, 3, 4, 22, 9, 4, 6, 5, 3, 3, 5, 3, 3, 4, 4, 4, 4, 3, 4, 3, 7, 3, 7, 3, 3, 7, 3, 4, 8, 4, 3, 5, 5, 12, 3, 3, 5, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 9, 7, 3, 7, 5, 5, 5, 3, 4, 3, 5, 3, 3, 4, 5, 8, 4, 3, 3, 7, 12, 3, 3, 3, 5, 3, 7, 3, 7, 3, 4, 3, 3, 3, 3, 4, 3, 5, 4, 4, 5, 6, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [12]:
# Cognition Layer
gamma_c = 3.0  # Power-law exponent
kmin_c = 3  # Minimum degree
ldeg_c = generate_degree_sequence(n, gamma_c, kmin_c)
print("Behavior Degree Sequence: ", ldeg_c)
cnw = nx.configuration_model(ldeg_c)
frac_prot = 0
lprot = assign_protection(cnw, frac_prot)

Behavior Degree Sequence:  [4, 3, 3, 3, 3, 3, 4, 6, 4, 3, 4, 7, 9, 3, 4, 3, 7, 11, 11, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 3, 3, 5, 5, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 6, 10, 3, 3, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 3, 10, 6, 3, 4, 4, 4, 3, 3, 7, 3, 3, 4, 3, 3, 6, 3, 4, 8, 3, 6, 4, 5, 3, 3, 4, 5, 3, 3, 3, 3, 3, 3, 3, 13, 3, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, 3, 5, 6, 4, 3, 3, 6, 3, 3, 4, 3, 3, 3, 11, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [19]:
# Epidemic Layer
gamma_e = 4.0 #gamma_e = 4.75   #gamma_e = 4.0
kmin_e = 3
ldeg_e = generate_degree_sequence(n, gamma_e, kmin_e)
print("Degree Sequence: ", ldeg_e)
enw = nx.configuration_model(ldeg_e)
k = ((gamma_e-1)/(gamma_e-2))*(kmin_e) #<k>
k2 =((gamma_e-1)/(gamma_e-3))*((kmin_e)**2) #<k^2>
division_factor = (k2-k)/k # division factor to compute beta_max(<k^2>-<k>)/<k>
print(["<k>: ", k, "<k^2>: ", k2,"(<k^2>-<k>)/<k>: ",  division_factor])

Degree Sequence:  [3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 8, 6, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 5, 4, 4, 4, 3, 3, 4, 3, 3, 3, 3, 8, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [20]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [21]:
def ICE_vaccine_record_severe(inw, ldeg_i, ltre, cnw, ldeg_c, lprot, enw, ldeg_e, lam, alp, omega, sigma, zeta_1, zeta_2, zeta_3, zeta_4, beta_VV, beta_NV, beta_VN, beta_NN, mu, n_sample):
  t_max = 1000      # Set maximum time
  kmax_i = max (ldeg_i)     # Get maximum hyperedge degree in information layer
  kmax_c = max (ldeg_c)     # Get maximum hyperedge degree in cognition layer
  kmax_e = max (ldeg_e)     # Get maximum degree in epidemic layer
  N = inw.order()  # Get the network size

  rho_A = []   # Keep track of fraction of stifler in information layer
  rho_V = []   # Keep track of fraction of protected in cognition layer
  rho_R = []   # Keep track of fraction of recovered in epidemic layer

  for i_samp in range(1, n_sample + 1):
      t = 0                 # Initialize time, number of corrected, number of recovered
      N_corrected = 0
      N_recovered = 0
      N_stifler = 0

      info_states = {j: "U" for j in inw.nodes()}   # Initialize information and disease states
      disease_states = {k: "S" for k in enw.nodes()}

      lprot = assign_protection(cnw, frac_prot)
      vaccinated = list(filter(lambda node: lprot[node] == "V", lprot))
      N_vaccinated = len(vaccinated)
      not_vaccinated = list(filter(lambda node: lprot[node] == "NV", lprot))

      gossip = []     # Create lists to store gossip and corrected individuals in information layer
      corrected = []
      stifler = []

      N_gossip = 0
      N_e_i = 0

      infected = []     # Create lists to store infected and recovered individuals in epidemic layer
      recovered = []
      severe = []

      time_record = [0]
      gossip_count = [0]
      vaccinated_count = [N_vaccinated]
      prevalence_count = [0]
      recovered_count = [0]
      severe_count = [0]

      N_infected = 0
      N_severe = 0
      N_e_e = 0
      for _ in range(5):
          ill_node_0 = np.random.choice([node for node, state in disease_states.items() if state == "S" and node in enw.nodes()])
          disease_states[ill_node_0] = "I"
          infected.append(ill_node_0)
          N_infected += 1
          N_e_e += enw.degree(ill_node_0)

      beta_max = max(beta_VV,beta_NV,beta_VN,beta_NN) # Chose the largest infection rate (I known that it is beta_NN, but keep it general)

      while t < 150 and N_infected > 0:   # We stop when there is no infection
        total_rate = lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N-N_vaccinated) + zeta_3 * N_vaccinated + zeta_4 * (N-N_vaccinated) + omega * N_stifler + sigma * N_gossip
        tau = -np.log(np.random.uniform(1e-6, 1)) / total_rate
        t += tau

        if t >= t_max:
                break

      # Determine which event occurs
        event = np.random.uniform()
        p1 = (lam * N_e_i) / total_rate     # rumor spreading
        p2 = (lam * N_e_i + alp * N_e_i) / total_rate  # rumor stifling (by meeting stifling neighbor threshold)
        p3 = (lam * N_e_i + 2 * alp * N_e_i) / total_rate  # rumor stifling (by meeting gossip neighbor threshold)

        p4 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e) / total_rate  # disease propagation
        p5 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected) / total_rate  # disease recovery

        p6 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated) / total_rate # change to not adopting protection by information
        p7 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N-N_vaccinated) ) / total_rate # change to adopting protection by information
        p8 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N-N_vaccinated)  + zeta_3 * N_vaccinated) / total_rate # change to not adopting protection by neighborhood behavior
        p9 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N-N_vaccinated) + zeta_3 * N_vaccinated + zeta_4 * (N-N_vaccinated))/total_rate # change to adopting protection by neighborhood behavior
        p10 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N-N_vaccinated) + zeta_3 * N_vaccinated + zeta_4 * (N-N_vaccinated) + omega * N_stifler)/total_rate # rumor interest renewal
        # >p10 correction of rummor


        # Determine if accept selected individual based on degree distribution
        q_deg_i = np.random.uniform()
        q_deg_c = np.random.uniform()
        q_deg_e = np.random.uniform()

        # Case 1: Rumor spreading
        if event < p1:
                response='F'
                while len(gossip)>0 and response =='F':
                  gossip_node = np.random.choice(gossip) #chose an infected node proportional to the degree
                  draw_rn = np.random.uniform()
                  if draw_rn < inw.degree(gossip_node)/kmax_i:
                     response='T'

                edges_containing_gossip = [edge for edge in inw.edges() if gossip_node in inw[edge]]
                if edges_containing_gossip:
                        rumor_hyper_edge = np.random.choice(edges_containing_gossip)
                        neighbors = inw[rumor_hyper_edge]
                        for neighbor in neighbors:
                            if info_states[neighbor] == "U":
                                count_gossip_neighbors = sum(1 for node in inw.neighbors(neighbor) if info_states[node] == "G")
                                if count_gossip_neighbors / len(inw.neighbors(neighbor)) >= ltre[neighbor]:
                                    info_states[neighbor] = "G"  # uninformed neighbor becomes gossip spreader
                                    gossip.append(neighbor)
                                    N_gossip += 1
                                    N_e_i += inw.degree(neighbor)
                                    time_record.append(t)
                                    gossip_count.append(N_gossip)
                                    vaccinated_count.append(N_vaccinated)
                                    prevalence_count.append(N_infected)
                                    recovered_count.append(N_recovered)
                                    severe_count.append(N_severe)

        # Case 2: Rumor stifling (by meeting stifling neighbor threshold)
        elif event < p2:
                    response='F'
                    while len(gossip)>0 and response =='F':
                      stifler_node = np.random.choice(gossip) #chose an infected node proportional to the degree
                      draw_rn = np.random.uniform()
                      if draw_rn < inw.degree(stifler_node)/kmax_i:
                         response='T'

                    stifler_hyper_edge = np.random.choice(list(inw.edges()))
                    neighbors = inw[stifler_hyper_edge]
                    count_stifler_neighbors = sum(1 for node in inw.neighbors(stifler_node) if info_states[node] == "C")
                    if count_stifler_neighbors > 0:
                            info_states[stifler_node] = "A"
                            N_gossip -= 1
                            gossip.remove(stifler_node)
                            stifler.append(stifler_node)
                            N_stifler += 1
                            N_e_i -= inw.degree(stifler_node)
                            time_record.append(t)
                            gossip_count.append(N_gossip)
                            vaccinated_count.append(N_vaccinated)
                            prevalence_count.append(N_infected)
                            recovered_count.append(N_recovered)
                            severe_count.append(N_severe)

        # Case 3: Rumor stifling (by meeting gossip neighbor threshold)
        elif event < p3:
                    response='F'
                    while len(gossip)>0 and response =='F':
                      stifler_node = np.random.choice(gossip) #chose an infected node proportional to the degree
                      draw_rn = np.random.uniform()
                      if draw_rn < inw.degree(stifler_node)/kmax_i:
                         response='T'

                    stifler_hyper_edge = np.random.choice(list(inw.edges()))
                    neighbors = inw[stifler_hyper_edge]
                    count_gossip_neighbors = sum(1 for node in inw.neighbors(stifler_node) if info_states[node] == "G")
                    if count_gossip_neighbors > 0:
                            info_states[stifler_node] = "A"
                            N_gossip -= 1
                            gossip.remove(stifler_node)
                            stifler.append(stifler_node)
                            N_stifler += 1
                            N_e_i -= inw.degree(stifler_node)
                            time_record.append(t)
                            gossip_count.append(N_gossip)
                            vaccinated_count.append(N_vaccinated)
                            prevalence_count.append(N_infected)
                            recovered_count.append(N_recovered)
                            severe_count.append(N_severe)

        # Case 4: Disease propagation
        elif event < p4:
            # code based on the new pseudo code
            response='F'
            while len(infected)>0 and response =='F':  #draw node until degree distribution is reached and while the infected list is not empty
              infected_node = np.random.choice(infected) #choose an infected node proportional to the degree
              draw_rn = np.random.uniform()
              if draw_rn < enw.degree(infected_node)/kmax_e: # kmax_e is the global max degree
                 response='T'

            neighbors = list(enw.neighbors(infected_node)) #get the list of neighbors of the infected node
            target_node = np.random.choice(neighbors) #choose a neighbor at random
            # beta_NN > beta_VN > beta_NV > beta_VV
            if disease_states[target_node] == "S":
                if lprot[target_node] == "V" and lprot[infected_node] == "V":
                   beta_correct = beta_VV
                elif lprot[target_node] == "NV" and lprot[infected_node] == "V":
                   beta_correct = beta_NV
                elif lprot[target_node] == "V" and lprot[infected_node] == "NV":
                   beta_correct = beta_VN
                else:
                   beta_correct = beta_NN

                draw_rn = np.random.uniform()
                if draw_rn < beta_correct / beta_max:
                      if lprot[target_node] == "V":
                        severe_prob = 0.01  # 1% severe if vaccinated
                      else:
                        severe_prob = 0.20  # 20% severe if unvaccinated

                      draw_rn_severe = np.random.uniform()
                      if draw_rn_severe < severe_prob:
                          disease_states[target_node] = "I_severe"  # Track severe cases
                          severe.append(target_node)
                          N_severe += 1
                      else:
                          disease_states[target_node] = "I"    # Track mild cases
                          infected.append(target_node)
                          N_infected += 1

                      N_e_e += enw.degree(target_node)
                      time_record.append(t)
                      gossip_count.append(N_gossip)
                      vaccinated_count.append(N_vaccinated)
                      prevalence_count.append(N_infected)
                      recovered_count.append(N_recovered)
                      severe_count.append(N_severe)

                   ## Start gossip after epidemic have started
                      if N_gossip < 10:
                        uninformed = [node for node, state in info_states.items() if state == "U" and node in inw.nodes()]
                        if len(uninformed) > 0:
                          rumor_node_0 = np.random.choice(uninformed)
                          info_states[rumor_node_0] = "G"
                          gossip.append(rumor_node_0)
                          N_gossip += 1
                          N_e_i += inw.degree(rumor_node_0)
                          time_record.append(t)
                          gossip_count.append(N_gossip)
                          vaccinated_count.append(N_vaccinated)
                          prevalence_count.append(N_infected)
                          recovered_count.append(N_recovered)
                          severe_count.append(N_severe)

        # Case 5: Disease recovery
        elif event < p5:
                  recovered_node = np.random.choice(infected)
                  disease_states[recovered_node] = "R"
                  infected.remove(recovered_node)
                  recovered.append(recovered_node)
                  N_infected -= 1
                  N_recovered += 1
                  N_e_e -= enw.degree(recovered_node)
                  time_record.append(t)
                  gossip_count.append(N_gossip)
                  vaccinated_count.append(N_vaccinated)
                  prevalence_count.append(N_infected)
                  recovered_count.append(N_recovered)
                  severe_count.append(N_severe)

        # Case 6: # Change to not adopting protection based on information layer
        # n_G is the total spreader neighbors on the information layer,
        # while k_info is the total neighbor count on the information layer
        elif event < p6:
            if len(vaccinated) > 0:
              node_to_not_protect = np.random.choice(vaccinated)
              n_G = sum(1 for node in filter(lambda x: info_states[x] == "G", inw.neighbors(node_to_not_protect)))
              k_info = len(list(inw.neighbors(node_to_not_protect)))
              frac_inf = N_infected/N
              b = 0.5
              if k_info > 0 and np.random.uniform() < (n_G / k_info)* (1-np.tanh(b*(frac_inf))):
                    lprot[node_to_not_protect] = "NV"
                    vaccinated.remove(node_to_not_protect)
                    not_vaccinated.append(node_to_not_protect)
                    N_vaccinated -= 1
                    time_record.append(t)
                    gossip_count.append(N_gossip)
                    vaccinated_count.append(N_vaccinated)
                    prevalence_count.append(N_infected)
                    recovered_count.append(N_recovered)
                    severe_count.append(N_severe)

        # Case 7: Change to adopting protection based on information layer
        elif event < p7:
            if len(not_vaccinated) > 0:
                node_to_protect = np.random.choice(not_vaccinated)
                n_G = sum(1 for node in filter(lambda x: info_states[x] == "G", inw.neighbors(node_to_protect)))
                n_C = sum(1 for node in filter(lambda x: info_states[x] == "C", inw.neighbors(node_to_protect)))
                k_info = len(list(inw.neighbors(node_to_protect)))
                if k_info > 0:
                  rho_G = n_G / k_info
                  rho_C = n_C / k_info
                  eta = 2
                  rho_I = N_infected/N
                  b = 0.5
                  if np.random.uniform() < (1 - rho_G + eta * rho_C)* np.tanh(b*(rho_I)):
                        lprot[node_to_protect] = "V"
                        not_vaccinated.remove(node_to_protect)
                        vaccinated.append(node_to_protect)
                        N_vaccinated += 1
                        time_record.append(t)
                        gossip_count.append(N_gossip)
                        vaccinated_count.append(N_vaccinated)
                        prevalence_count.append(N_infected)
                        recovered_count.append(N_recovered)
                        severe_count.append(N_severe)


        # Case 8: # Change to not adopting protection based on neighborhood behavior in cognition layer
        # n_P is the total protected neighbors on the cognition layer,
        # while k_cog is the total neighbor count on the cognition layer
        elif event < p8:
            if len(vaccinated) > 0:
                node_to_not_protect = np.random.choice(vaccinated)
                n_P = sum(1 for node in filter(lambda x: lprot[x] == "V", cnw.neighbors(node_to_not_protect)))
                k_cog = len(list(cnw.neighbors(node_to_not_protect)))
                if k_cog > 0 and np.random.uniform() < 1 - n_P / k_cog:
                        lprot[node_to_not_protect] = "NV"
                        vaccinated.remove(node_to_not_protect)
                        not_vaccinated.append(node_to_not_protect)
                        N_vaccinated -= 1
                        time_record.append(t)
                        gossip_count.append(N_gossip)
                        vaccinated_count.append(N_vaccinated)
                        prevalence_count.append(N_infected)
                        recovered_count.append(N_recovered)
                        severe_count.append(N_severe)


        # Case 9: # Change to adopting protection based on neighborhood behavior in cognition layer
        elif event < p9:
            if len(not_vaccinated) > 0:
                node_to_protect = np.random.choice(not_vaccinated)
                n_P = sum(1 for node in filter(lambda x: lprot[x] == "V", cnw.neighbors(node_to_protect)))
                k_cog = len(list(cnw.neighbors(node_to_protect)))
                if k_cog > 0 and np.random.uniform() < n_P / k_cog:
                        lprot[node_to_protect] = "V"
                        not_vaccinated.remove(node_to_protect)
                        vaccinated.append(node_to_protect)
                        N_vaccinated += 1
                        time_record.append(t)
                        gossip_count.append(N_gossip)
                        vaccinated_count.append(N_vaccinated)
                        prevalence_count.append(N_infected)
                        recovered_count.append(N_recovered)
                        severe_count.append(N_severe)

        # Case 10: Rumor interest renewal
        elif event < p10:
            if len(stifler) > 0:
                  stifler_node = np.random.choice(stifler)
                  info_states[stifler_node] = "U"
                  stifler.remove(stifler_node)
                  N_stifler -= 1

        # Case 11: Correction of rumor
        else:
            if len(gossip)>0:
                correction_node = np.random.choice(gossip) #chose an infected node proportional to the degree
                info_states[correction_node] = "C"
                N_gossip -= 1
                gossip.remove(correction_node)
                corrected.append(correction_node)
                N_corrected += 1
                N_e_i -= inw.degree(correction_node)
                time_record.append(t)
                gossip_count.append(N_gossip)
                vaccinated_count.append(N_vaccinated)
                prevalence_count.append(N_infected)
                recovered_count.append(N_recovered)
                severe_count.append(N_severe)

  return time_record, gossip_count, vaccinated_count, recovered_count, severe_count

In [22]:
def ICE_vaccine_record_hub_severe(inw, ldeg_i, ltre, cnw, ldeg_c, lprot, enw, ldeg_e, lam, alp, omega, sigma, zeta_1, zeta_2, zeta_3, zeta_4, beta_VV, beta_NV, beta_VN, beta_NN, mu, n_sample):
    t_max = 1000      # Set maximum time
    kmax_i = max(ldeg_i)     # Get maximum hyperedge degree in information layer
    kmax_c = max(ldeg_c)     # Get maximum hyperedge degree in cognition layer
    kmax_e = max(ldeg_e)     # Get maximum degree in epidemic layer
    N = inw.order()  # Get the network size

    rho_A = []   # Keep track of fraction of stifler in information layer
    rho_V = []   # Keep track of fraction of protected in cognition layer
    rho_R = []   # Keep track of fraction of recovered in epidemic layer

    for i_samp in range(1, n_sample + 1):
        t = 0                 # Initialize time, number of corrected, number of recovered
        N_corrected = 0
        N_recovered = 0
        N_stifler = 0

        info_states = {j: "U" for j in inw.nodes()}   # Initialize information and disease states
        disease_states = {k: "S" for k in enw.nodes()}

        lprot = assign_protection(cnw, frac_prot)
        vaccinated = list(filter(lambda node: lprot[node] == "V", lprot))
        N_vaccinated = len(vaccinated)
        not_vaccinated = list(filter(lambda node: lprot[node] == "NV", lprot))

        gossip = []     # Create lists to store gossip and corrected individuals in information layer
        corrected = []
        stifler = []

        N_gossip = 0
        N_e_i = 0

        infected = []     # Create lists to store infected and recovered individuals in epidemic layer
        recovered = []
        severe = []

        time_record = [0]
        gossip_count = [0]
        vaccinated_count = [N_vaccinated]
        prevalence_count = [0]
        recovered_count = [0]
        severe_count = [0]

        N_infected = 0
        N_severe = 0
        N_e_e = 0
        for _ in range(5):
            ill_node_0 = np.random.choice([node for node, state in disease_states.items() if state == "S" and node in enw.nodes()])
            disease_states[ill_node_0] = "I"
            infected.append(ill_node_0)
            N_infected += 1
            N_e_e += enw.degree(ill_node_0)

        beta_max = max(beta_VV, beta_NV, beta_VN, beta_NN)  # Chose the largest infection rate (I known that it is beta_NN, but keep it general)

        while t < 150 and N_infected > 0:   # We stop when there is no infection
            total_rate = lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N - N_vaccinated) + zeta_3 * N_vaccinated + zeta_4 * (N - N_vaccinated) + omega * N_stifler + sigma * N_gossip
            tau = -np.log(np.random.uniform(1e-6, 1)) / total_rate
            t += tau

            if t >= t_max:
                break

            # Determine which event occurs
            event = np.random.uniform()
            p1 = (lam * N_e_i) / total_rate     # rumor spreading
            p2 = (lam * N_e_i + alp * N_e_i) / total_rate  # rumor stifling (by meeting stifling neighbor threshold)
            p3 = (lam * N_e_i + 2 * alp * N_e_i) / total_rate  # rumor stifling (by meeting gossip neighbor threshold)

            p4 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e) / total_rate  # disease propagation
            p5 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected) / total_rate  # disease recovery

            p6 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated) / total_rate # change to not adopting protection by information
            p7 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N - N_vaccinated)) / total_rate # change to adopting protection by information
            p8 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N - N_vaccinated) + zeta_3 * N_vaccinated) / total_rate # change to not adopting protection by neighborhood behavior
            p9 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N - N_vaccinated) + zeta_3 * N_vaccinated + zeta_4 * (N - N_vaccinated)) / total_rate # change to adopting protection by neighborhood behavior
            p10 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_vaccinated + zeta_2 * (N - N_vaccinated) + zeta_3 * N_vaccinated + zeta_4 * (N - N_vaccinated) + omega * N_stifler) / total_rate # rumor interest renewal
            # >p10 correction of rummor

            # Determine if accept selected individual based on degree distribution
            q_deg_i = np.random.uniform()
            q_deg_c = np.random.uniform()
            q_deg_e = np.random.uniform()

            # Case 1: Rumor spreading
            if event < p1:
                response = 'F'
                while len(gossip) > 0 and response == 'F':
                    gossip_node = np.random.choice(gossip)  # chose an infected node proportional to the degree
                    draw_rn = np.random.uniform()
                    if draw_rn < inw.degree(gossip_node) / kmax_i:
                        response = 'T'

                edges_containing_gossip = [edge for edge in inw.edges() if gossip_node in inw[edge]]
                if edges_containing_gossip:
                    rumor_hyper_edge = np.random.choice(edges_containing_gossip)
                    neighbors = inw[rumor_hyper_edge]
                    for neighbor in neighbors:
                        if info_states[neighbor] == "U":
                            count_gossip_neighbors = sum(1 for node in inw.neighbors(neighbor) if info_states[node] == "G")
                            if count_gossip_neighbors / len(inw.neighbors(neighbor)) >= ltre[neighbor]:
                                info_states[neighbor] = "G"  # uninformed neighbor becomes gossip spreader
                                gossip.append(neighbor)
                                N_gossip += 1
                                N_e_i += inw.degree(neighbor)
                                time_record.append(t)
                                gossip_count.append(N_gossip)
                                vaccinated_count.append(N_vaccinated)
                                prevalence_count.append(N_infected)
                                recovered_count.append(N_recovered)
                                severe_count.append(N_severe)


            # Case 2: Rumor stifling (by meeting stifling neighbor threshold)
            elif event < p2:
                response = 'F'
                while len(gossip) > 0 and response == 'F':
                    stifler_node = np.random.choice(gossip)  # chose an infected node proportional to the degree
                    draw_rn = np.random.uniform()
                    if draw_rn < inw.degree(stifler_node) / kmax_i:
                        response = 'T'

                stifler_hyper_edge = np.random.choice(list(inw.edges()))
                neighbors = inw[stifler_hyper_edge]
                count_stifler_neighbors = sum(1 for node in inw.neighbors(stifler_node) if info_states[node] == "C")
                if count_stifler_neighbors > 0:
                    info_states[stifler_node] = "A"
                    N_gossip -= 1
                    gossip.remove(stifler_node)
                    stifler.append(stifler_node)
                    N_stifler += 1
                    N_e_i -= inw.degree(stifler_node)
                    time_record.append(t)
                    gossip_count.append(N_gossip)
                    vaccinated_count.append(N_vaccinated)
                    prevalence_count.append(N_infected)
                    recovered_count.append(N_recovered)
                    severe_count.append(N_severe)

            # Case 3: Rumor stifling (by meeting gossip neighbor threshold)
            elif event < p3:
                response = 'F'
                while len(gossip) > 0 and response == 'F':
                    stifler_node = np.random.choice(gossip)  # chose an infected node proportional to the degree
                    draw_rn = np.random.uniform()
                    if draw_rn < inw.degree(stifler_node) / kmax_i:
                        response = 'T'

                stifler_hyper_edge = np.random.choice(list(inw.edges()))
                neighbors = inw[stifler_hyper_edge]
                count_gossip_neighbors = sum(1 for node in inw.neighbors(stifler_node) if info_states[node] == "G")
                if count_gossip_neighbors > 0:
                    info_states[stifler_node] = "A"
                    N_gossip -= 1
                    gossip.remove(stifler_node)
                    stifler.append(stifler_node)
                    N_stifler += 1
                    N_e_i -= inw.degree(stifler_node)
                    time_record.append(t)
                    gossip_count.append(N_gossip)
                    vaccinated_count.append(N_vaccinated)
                    prevalence_count.append(N_infected)
                    recovered_count.append(N_recovered)
                    severe_count.append(N_severe)

            # Case 4: Disease propagation
            elif event < p4:
                # code based on the new pseudo code
                response = 'F'
                while len(infected) > 0 and response == 'F':  # draw node until degree distribution is reached and while the infected list is not empty
                    infected_node = np.random.choice(infected)  # choose an infected node proportional to the degree
                    draw_rn = np.random.uniform()
                    if draw_rn < enw.degree(infected_node) / kmax_e:  # kmax_e is the global max degree
                        response = 'T'

                neighbors = list(enw.neighbors(infected_node))  # get the list of neighbors of the infected node
                target_node = np.random.choice(neighbors)  # choose a neighbor at random
                # beta_NN > beta_VN > beta_NV > beta_VV
                if disease_states[target_node] == "S":
                    if lprot[target_node] == "V" and lprot[infected_node] == "V":
                        beta_correct = beta_VV
                    elif lprot[target_node] == "NV" and lprot[infected_node] == "V":
                        beta_correct = beta_NV
                    elif lprot[target_node] == "V" and lprot[infected_node] == "NV":
                        beta_correct = beta_VN
                    else:
                        beta_correct = beta_NN

                    draw_rn = np.random.uniform()
                    if draw_rn < beta_correct / beta_max:
                      if lprot[target_node] == "V":
                        severe_prob = 0.01  # 1% severe if vaccinated
                      else:
                        severe_prob = 0.20  # 20% severe if unvaccinated

                      draw_rn_severe = np.random.uniform()
                      if draw_rn_severe < severe_prob:
                          disease_states[target_node] = "I_severe"  # Track severe cases
                          severe.append(target_node)
                          N_severe += 1
                      else:
                          disease_states[target_node] = "I"    # Track mild cases
                          infected.append(target_node)
                          N_infected += 1

                      N_e_e += enw.degree(target_node)
                      time_record.append(t)
                      gossip_count.append(N_gossip)
                      vaccinated_count.append(N_vaccinated)
                      prevalence_count.append(N_infected)
                      recovered_count.append(N_recovered)
                      severe_count.append(N_severe)

                      ## Start gossip after epidemic have started
                      if N_gossip < 10:
                            uninformed = [node for node, state in info_states.items() if state == "U" and node in inw.nodes()]
                            if len(uninformed) > 0:
                                rumor_node_0 = np.random.choice(uninformed)
                                info_states[rumor_node_0] = "G"
                                gossip.append(rumor_node_0)
                                N_gossip += 1
                                N_e_i += inw.degree(rumor_node_0)
                                time_record.append(t)
                                gossip_count.append(N_gossip)
                                vaccinated_count.append(N_vaccinated)
                                prevalence_count.append(N_infected)
                                recovered_count.append(N_recovered)
                                severe_count.append(N_severe)

            # Case 5: Disease recovery
            elif event < p5:
                recovered_node = np.random.choice(infected)
                disease_states[recovered_node] = "R"
                infected.remove(recovered_node)
                recovered.append(recovered_node)
                N_infected -= 1
                N_recovered += 1
                N_e_e -= enw.degree(recovered_node)
                time_record.append(t)
                gossip_count.append(N_gossip)
                vaccinated_count.append(N_vaccinated)
                prevalence_count.append(N_infected)
                recovered_count.append(N_recovered)
                severe_count.append(N_severe)

            # Case 6: # Change to not adopting protection based on information layer
            # n_G is the total spreader neighbors on the information layer,
            # while k_info is the total neighbor count on the information layer
            elif event < p6:
                if len(vaccinated) > 0:
                    node_to_not_protect = np.random.choice(vaccinated)
                    n_G = sum(1 for node in filter(lambda x: info_states[x] == "G", inw.neighbors(node_to_not_protect)))
                    k_info = len(list(inw.neighbors(node_to_not_protect)))
                    frac_inf = N_infected / N
                    b = 0.5
                    if k_info > 0 and np.random.uniform() < (n_G / k_info) * (1 - np.tanh(b * (frac_inf))):
                        lprot[node_to_not_protect] = "NV"
                        vaccinated.remove(node_to_not_protect)
                        not_vaccinated.append(node_to_not_protect)
                        N_vaccinated -= 1
                        time_record.append(t)
                        gossip_count.append(N_gossip)
                        vaccinated_count.append(N_vaccinated)
                        prevalence_count.append(N_infected)
                        recovered_count.append(N_recovered)
                        severe_count.append(N_severe)

            # Case 7: Change to adopting protection based on information layer (HUB PRIORITIZATION)
            elif event < p7:
                if len(not_vaccinated) > 0:
                    # Find all nodes with maximum degree and vaccinate one of them
                    max_degree = max(enw.degree(node) for node in not_vaccinated)
                    candidates = [node for node in not_vaccinated if enw.degree(node) == max_degree]
                    node_to_protect = np.random.choice(candidates)

                    n_G = sum(1 for node in filter(lambda x: info_states[x] == "G", inw.neighbors(node_to_protect)))
                    n_C = sum(1 for node in filter(lambda x: info_states[x] == "C", inw.neighbors(node_to_protect)))
                    k_info = len(list(inw.neighbors(node_to_protect)))
                    if k_info > 0:
                        rho_G = n_G / k_info
                        rho_C = n_C / k_info
                        eta = 2
                        rho_I = N_infected / N
                        b = 0.5
                        if np.random.uniform() < (1 - rho_G + eta * rho_C) * np.tanh(b * (rho_I)):
                            lprot[node_to_protect] = "V"
                            not_vaccinated.remove(node_to_protect)
                            vaccinated.append(node_to_protect)
                            N_vaccinated += 1
                            time_record.append(t)
                            gossip_count.append(N_gossip)
                            vaccinated_count.append(N_vaccinated)
                            prevalence_count.append(N_infected)
                            recovered_count.append(N_recovered)
                            severe_count.append(N_severe)

            # Case 8: # Change to not adopting protection based on neighborhood behavior in cognition layer
            # n_P is the total protected neighbors on the cognition layer,
            # while k_cog is the total neighbor count on the cognition layer
            elif event < p8:
                if len(vaccinated) > 0:
                    node_to_not_protect = np.random.choice(vaccinated)
                    n_P = sum(1 for node in filter(lambda x: lprot[x] == "V", cnw.neighbors(node_to_not_protect)))
                    k_cog = len(list(cnw.neighbors(node_to_not_protect)))
                    if k_cog > 0 and np.random.uniform() < 1 - n_P / k_cog:
                        lprot[node_to_not_protect] = "NV"
                        vaccinated.remove(node_to_not_protect)
                        not_vaccinated.append(node_to_not_protect)
                        N_vaccinated -= 1
                        time_record.append(t)
                        gossip_count.append(N_gossip)
                        vaccinated_count.append(N_vaccinated)
                        prevalence_count.append(N_infected)
                        recovered_count.append(N_recovered)
                        severe_count.append(N_severe)

            # Case 9: # Change to adopting protection based on neighborhood behavior in cognition layer (HUB PRIORITIZATION)
            elif event < p9:
                if len(not_vaccinated) > 0:
                    max_degree = max(enw.degree(node) for node in not_vaccinated)
                    candidates = [node for node in not_vaccinated if enw.degree(node) == max_degree]
                    node_to_protect = np.random.choice(candidates)
                    n_P = sum(1 for node in filter(lambda x: lprot[x] == "V", cnw.neighbors(node_to_protect)))
                    k_cog = len(list(cnw.neighbors(node_to_protect)))
                    if k_cog > 0 and np.random.uniform() < n_P / k_cog:
                        lprot[node_to_protect] = "V"
                        not_vaccinated.remove(node_to_protect)
                        vaccinated.append(node_to_protect)
                        N_vaccinated += 1
                        time_record.append(t)
                        gossip_count.append(N_gossip)
                        vaccinated_count.append(N_vaccinated)
                        prevalence_count.append(N_infected)
                        recovered_count.append(N_recovered)
                        severe_count.append(N_severe)

            # Case 10: Rumor interest renewal
            elif event < p10:
                if len(stifler) > 0:
                    stifler_node = np.random.choice(stifler)
                    info_states[stifler_node] = "U"
                    stifler.remove(stifler_node)
                    N_stifler -= 1

            # Case 11: Correction of rumor
            else:
                if len(gossip) > 0:
                    correction_node = np.random.choice(gossip)  # chose an infected node proportional to the degree
                    info_states[correction_node] = "C"
                    N_gossip -= 1
                    gossip.remove(correction_node)
                    corrected.append(correction_node)
                    N_corrected += 1
                    N_e_i -= inw.degree(correction_node)
                    time_record.append(t)
                    gossip_count.append(N_gossip)
                    vaccinated_count.append(N_vaccinated)
                    prevalence_count.append(N_infected)
                    recovered_count.append(N_recovered)
                    severe_count.append(N_severe)

    return time_record, gossip_count, vaccinated_count, recovered_count, severe_count

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Parameters
lam = 1/3  # gossip spreading rate
alp = 1/5  # gossip stifling rate
omega = 1/3  # gossip renewal rate
sigma = 0.05  # correction rate of misinformation

zeta_1 = 0  # go to not protect by information
zeta_3 = 0  # go to not protect by neighborhood behavior
zeta_2 = 0.10  # go to protect by information
zeta_4 = 0  # go to protect by neighborhood behavior

mu = 0.2
R0 = 4.0  # reproduction number
beta_NN = R0 * mu / division_factor
beta_VN = beta_NN * 0.2
beta_NV = beta_NN * 0.1
beta_VV = beta_NN * 0
n_sample = 1
N = 500  # Network size
n_simulations = 100

plt.rcParams.update({
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'axes.titlesize': 14
})

# Define strategies
strategies = {
    'random': ICE_vaccine_record_severe,
    'hub_priority': ICE_vaccine_record_hub_severe
}

# Create output directory if it doesn't exist
output_dir = '/content/drive/My Drive/Network_Model_Vaccines'
os.makedirs(output_dir, exist_ok=True)

# Generate and store all simulation data
all_results = {}
for strategy_name, strategy_func in strategies.items():
    print(f"Running {strategy_name} strategy simulations...")

    # Initialize storage for this strategy
    strategy_data = pd.DataFrame()

    for i in range(n_simulations):
        time, gossip, vaccinated, recovered, severe = strategy_func(
            inw, ldeg_i, ltre, cnw, ldeg_c, lprot, enw, ldeg_e,
            lam, alp, omega, sigma, zeta_1, zeta_2, zeta_3, zeta_4,
            beta_VV, beta_NV, beta_VN, beta_NN, mu, n_sample)

        # Append to strategy DataFrame
        run_df = pd.DataFrame({
            'time': time,
            'gossip': gossip,
            'vaccinated': vaccinated,
            'recovered': recovered,
            'severe': severe,
            'run_id': i
        })
        strategy_data = pd.concat([strategy_data, run_df])

    # Save this strategy's data to CSV
    csv_path = os.path.join(output_dir, f'0516_severe_symptoms_info_vac_only_{strategy_name}_runs.csv')
    strategy_data.to_csv(csv_path, index=False)
    print(f"Saved {strategy_name} data to {csv_path}")

    # Store in all_results for plotting
    all_results[strategy_name] = {
        'time_records': [time for time in strategy_data.groupby('run_id')['time'].apply(list)],
        'gossip_counts': [gossip for gossip in strategy_data.groupby('run_id')['gossip'].apply(list)],
        'vaccinated_counts': [vaccinated for vaccinated in strategy_data.groupby('run_id')['vaccinated'].apply(list)],
        'recovered_counts': [recovered for recovered in strategy_data.groupby('run_id')['recovered'].apply(list)],
        'severe_counts': [severe for severe in strategy_data.groupby('run_id')['severe'].apply(list)]
    }



Running random strategy simulations...
Saved random data to /content/drive/My Drive/Network_Model_Vaccines/0516_severe_symptoms_info_vac_only_random_runs.csv
Running hub_priority strategy simulations...
Saved hub_priority data to /content/drive/My Drive/Network_Model_Vaccines/0516_severe_symptoms_info_vac_only_hub_priority_runs.csv


In [ ]:
# Process results for plotting
for strategy in strategies:
    max_time = max([max(t) for t in all_results[strategy]['time_records']])
    common_time = np.linspace(0, max_time, 150)
    all_results[strategy]['common_time'] = common_time

    # Initialize averages
    all_results[strategy]['avg_gossip'] = np.zeros(len(common_time))
    all_results[strategy]['avg_vaccinated'] = np.zeros(len(common_time))
    all_results[strategy]['avg_recovered'] = np.zeros(len(common_time))
    all_results[strategy]['avg_severe'] = np.zeros(len(common_time))

    # Calculate averages
    for i in range(n_simulations):
        all_results[strategy]['avg_gossip'] += np.interp(
            common_time,
            all_results[strategy]['time_records'][i],
            all_results[strategy]['gossip_counts'][i]
        )
        all_results[strategy]['avg_vaccinated'] += np.interp(
            common_time,
            all_results[strategy]['time_records'][i],
            all_results[strategy]['vaccinated_counts'][i]
        )
        all_results[strategy]['avg_recovered'] += np.interp(
            common_time,
            all_results[strategy]['time_records'][i],
            all_results[strategy]['recovered_counts'][i]
        )
        all_results[strategy]['avg_severe'] += np.interp(
            common_time,
            all_results[strategy]['time_records'][i],
            all_results[strategy]['severe_counts'][i]
        )

    # Final averages
    all_results[strategy]['avg_gossip'] /= n_simulations
    all_results[strategy]['avg_vaccinated'] /= n_simulations
    all_results[strategy]['avg_recovered'] /= n_simulations
    all_results[strategy]['avg_severe'] /= n_simulations

# Create the plot
fig, axes = plt.subplots(4, 2, figsize=(16, 16), sharex=True, sharey='row')

# Plot parameters
row_titles = ['Gossip Spreader Percentage (%)', 'Vaccinated Percentage (%)', 'Recovered Percentage (%)', 'Severe Percentage (%)']
row_colors = ['skyblue', 'peru', 'purple', 'red']
row_ylimits = [(0, 90), (0, 70), (0, 70),(0,20)]
panel_labels = ['A', 'B', 'C', 'D', 'E', 'F','G','H']
strategy_titles = ['Random Vaccination', 'Hub-Prioritized Vaccination']

# Create plots
for row in range(4):
    for col, strategy in enumerate(strategies):
        panel_index = row * 2 + col

        # Select data
        if row == 0:
            metric = 'gossip_counts'
            avg_data = all_results[strategy]['avg_gossip']/N*100
        elif row == 1:
            metric = 'vaccinated_counts'
            avg_data = all_results[strategy]['avg_vaccinated']/N*100
        elif row == 2:
            metric = 'recovered_counts'
            avg_data = all_results[strategy]['avg_recovered']/N*100
        else:
            metric = 'severe_counts'
            avg_data = all_results[strategy]['avg_severe']/N*100

        # Plot individual runs
        for i in range(n_simulations):
            axes[row, col].plot(
                all_results[strategy]['time_records'][i],
                np.array(all_results[strategy][metric][i])/N*100,
                alpha=0.1,
                color=row_colors[row]
            )

        # Plot average
        axes[row, col].plot(
            all_results[strategy]['common_time'],
            avg_data,
            color='black',
            linewidth=2
        )

        # Formatting
        axes[row, col].grid(True)
        axes[row, col].set_ylim(row_ylimits[row])

        # Add titles and labels
        if row == 0:
            axes[row, col].set_title(strategy_titles[col])
        if col == 0:
            axes[row, col].set_ylabel(row_titles[row])

        # Add panel labels
        axes[row, col].text(
            0.02, 0.98, panel_labels[panel_index],
            transform=axes[row, col].transAxes,
            fontsize=14, fontweight='bold',
            verticalalignment='top'
        )

# X-axis labels
for ax in axes[3, :]:
    ax.set_xlabel('Time')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, '0516_severe_symptoms_info_vac_only_compare_strategies_percentage.pdf'), format='pdf')
plt.show()